In [15]:
from transformers import GPT2Tokenizer
from main import NeuroFill, NeuroFillConfig
import torch
import torch.nn as nn
from torch.nn import functional as F

neurofill_config = NeuroFillConfig()
neurofill_model = NeuroFill(neurofill_config)
enc = GPT2Tokenizer.from_pretrained('gpt2') 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
neurofill_model.load_state_dict(torch.load('model_20000.pth'))
model = neurofill_model.to(device)

model.eval()
num_return_sequences = 4
max_length = 128
text = "Money is the root of all evil"
tokens = enc.encode(text)
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
xgen = tokens.to(device)
sample_rng = torch.Generator(device=device)
while xgen.size(1) < max_length:
    with torch.no_grad():
        # with torch.autocast(device_type=device, dtype=torch.bfloat16):
        logits, _ = model(xgen)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        ix = torch.multinomial(topk_probs, 1, generator=sample_rng)
        xcol = torch.gather(topk_indices, -1, ix)
        xgen = torch.cat((xgen, xcol), dim=1)
for i in range(num_return_sequences):
    tokens = xgen[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(f"\nsample {i}: \n{decoded}\n")


sample 0: 
Money is the root of all evil to a man as you do.<|endoftext|>As a bit of a second, when you say, The P:S. So I Am Allocation

And-

The 4, you've a lot of the "sail" the "hoor"

In the late 1980

It, the most years


In a one-seasonly-in-one, it shouldy, is a four-body-

When the two-managing for men, the more, I've the back and you are in. It is too. The "Hem-


sample 1: 
Money is the root of all evil. You may also, what's in the house-holding. It's not just a bad idea of your face-in-the-nour-as-it's-it, and by the body of your body-at-you-you-you-happing-your-in-you-they, you be- you might in the face of your face, who is in-onion that's-what are you, the the rest of the-body is a bad-body is your work, right-up. Why do you say this mayor (em-


sample 2: 
Money is the root of all evil. I'm a bit of a day after the future of the car, it's the best of a bit.

The more that you will have the "lone-all" - and a year.

"If You Go Round Your Business, You've Been Made.

Thi

In [3]:
neurofill_model

NeuroFill(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x BLOCK(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): AttentionNet(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)